In [22]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
from sklearn import linear_model
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, confusion_matrix

In [23]:
def readCSV(path):
    with open(path, 'r') as f:
        f.readline()
        for l in f:
            a, b, i = l.strip().split(',')
            i = int(i)
            yield a, b, i

In [24]:
allRatings = []
for l in readCSV("train.csv"):
    allRatings.append(l)

In [25]:
geneCount = defaultdict(int)
totalInteractions = 0
for geneA, geneB, interaction in readCSV("train.csv"):
    if interaction == 0: continue
    geneCount[geneA] += 1
    geneCount[geneB] += 1
    totalInteractions += 1

mostPopular = [(geneCount[x], x) for x in geneCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalInteractions/2:
        break

In [26]:
testRatings = []
for l in readCSV("test.csv"):
    testRatings.append(l)

In [27]:
def getAccuracy(yPred, yTrue):
    total = len(yPred)
    correct = 0
    for i in range(total):
        if yPred[i] == yTrue[i]:
            correct += 1

    return correct / total

In [28]:
yPred = []
yTrue = []
for geneA, geneB, interaction in testRatings:
    yTrue.append(interaction)
    if geneA in return1 or geneB in return1:
        yPred.append(1)
    else: yPred.append(0)

In [29]:
acc = getAccuracy(yPred=yPred, yTrue=yTrue)

In [30]:
acc

0.6490149164454746

In [31]:
def calcConfusionMatrix(y, yPred):
    tnfp, fntp = confusion_matrix(y, yPred)
    tn, fp = tnfp
    fn, tp = fntp

    # tp: number of times model predicts male correctly
    # tn: number of times model predicts female correctly

    # fp: number of times model predicts male when female
    # fn: number of times model predicts female when male

    # balanced ber rate:
    fpRate = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnRate = fn / (fn + tp) if (fn + tp) > 0 else 0
    ber = (fpRate + fnRate) / 2

    return [tp, tn, fp, fn, ber]

In [32]:
tp, tn, fp, fn, ber = calcConfusionMatrix(yTrue, yPred)

print(f"True P: {tp}")
print(f"True N: {tn}")
print(f"False P: {fp}")
print(f"False N: {fn}")
print(f"Ber: {tp}")

True P: 8507
True N: 26214
False P: 9519
False N: 9258
Ber: 8507
